In [1]:
# import socket and json
import socket
from time import sleep
import pickle

# Import required modules
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import time
import datetime

import re

import nltk
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/anaconda/envs/py36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
def GetAllTextFromContentBody(contentBody):
    out_str = ''
    for x in contentBody:
        out_str += x.text
        
    return out_str

In [3]:
def GetDateOfArticle(publishedData):
    time_, GMT = publishedData.split('+')
    dt = datetime.datetime.strptime(time_, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
    return dt

In [4]:
def GetDateTimeOfArticle(soup):
    metatags = soup.find_all('meta',attrs={'name':'publish-date'})
    for tag in metatags:
        return GetDateOfArticle(tag['content'])

In [5]:
def GetSentimentValueFromNLTK(articleText):
    
    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    
    if not articleText:
        return scores
    
    raw_text = articleText
    raw_text = re.sub("\n", ". ", str(raw_text))
    
    sid = SentimentIntensityAnalyzer()
    sentences = tokenize.sent_tokenize(raw_text)

    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    COUNT = 0
    for sentence in sentences:
        COUNT += 1
        ss = sid.polarity_scores(sentence)

        for k in sorted(ss):            
            scores[k] += ss[k]
            
    if COUNT != 0:        
        scores['pos'] /= COUNT
        scores['neu'] /= COUNT
        scores['neg'] /= COUNT
        scores['compound'] /= COUNT
        
        scores['pos'] = round(scores['pos'],2)
        scores['neu'] = round(scores['neu'],2)
        scores['neg'] = round(scores['neg'],2)
        scores['compound'] = round(scores['compound'],2)
        
    return scores

In [6]:
#def CheckCompanyNameInArticle(articleText):
    
#    if 'companyName' in articleText.lower().split():
#        return 1

#    return 0




In [7]:
def GetDataFromLinks(column):
        r = requests.get(x)
        html_content = r.text
        soup = BeautifulSoup(html_content , 'lxml')
        contentbody = soup.find_all("div", id=lambda value: value and value.startswith("content-body"))
        articleText = GetAllTextFromContentBody(contentbody)
                
        scores = GetSentimentValueFromNLTK(articleText)
        articleDate = GetDateTimeOfArticle(soup)
        
        seperator = ','
                
        #arr = ([str(articleDate.encode("utf-8"))],[str(score.encode("utf-8"))])
        #arr = ([str(articleDate)],[str(score)])
        #data_string = pickle.dumps(arr)                    
        #data_string = str(articleDate).encode('utf-8')+ str(seperator).encode('utf-8') + str(score).encode('utf-8') +'\n'
        
        data_string_Seq = (str(articleDate),str(scores['pos']),str(scores['neu']),str(scores['neg']),str(scores['compound']))
        data_string = (seperator.join( data_string_Seq ))
        return data_string + '\n'    

In [8]:
df = pd.read_csv('all_links.csv')
columnData = df['links']

with open('SentimentalScore.txt','a') as tf:
    for x in columnData:            
        dataFromLink = GetDataFromLinks(x)
        print (dataFromLink)
        tf.write(dataFromLink)
        tf.write('\n')
        
print("Completed")   

2016-02-20 17:27:35,0.06,0.76,0.04,0.03

2016-02-20 15:33:11,0.03,0.69,0.03,0.05

2016-02-20 14:00:12,0.06,0.72,0.07,-0.17

2016-02-20 15:04:47,0.06,0.87,0.02,0.05

Completed
